## RPFS Problem (TWCT objective) - Study Case - Tables and Graphs

Before running this, notebook, please run notebook 0.1.

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [ ]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [ ]:
# https://stackoverflow.com/questions/51483901/is-there-a-way-to-extend-the-solid-color-background-to-the-full-width-of-the-pag
class ExtendedTextBox(BoxStyle._Base):
    """
    An Extended Text Box that expands to the axes limits 
                        if set in the middle of the axes
    """

    def __init__(self, pad=0.3, width=500.):
        """
        width: 
            width of the textbox. 
            Use `ax.get_window_extent().width` 
                   to get the width of the axes.
        pad: 
            amount of padding (in vertical direction only)
        """
        self.width=width
        self.pad = pad
        super(ExtendedTextBox, self).__init__()

    def transmute(self, x0, y0, width, height, mutation_size):
        """
        x0 and y0 are the lower left corner of original text box
        They are set automatically by matplotlib
        """
        # padding
        pad = mutation_size * self.pad

        # we add the padding only to the box height
        height = height + 2.*pad
        # boundary of the padded box
        y0 = y0 - pad
        y1 = y0 + height
        _x0 = x0
        x0 = _x0 +width /2. - self.width/2.
        x1 = _x0 +width /2. + self.width/2.

        cp = [(x0, y0),
              (x1, y0), (x1, y1), (x0, y1),
              (x0, y0)]

        com = [Path.MOVETO,
               Path.LINETO, Path.LINETO, Path.LINETO,
               Path.CLOSEPOLY]

        path = Path(cp, com)

        return path

### List files in the result folder 

In [ ]:
# Main result file for RobPFSP-TWCT
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, 'RPFS_TWCT_all_results.pkl.gz')

In [ ]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
# Worstcase calculations for the robust and deterministic methods
rpfs_worstcase_file = os.path.join('..', 'pfsp_experiments', 'calculate_rpfs_twct_worstcase', 
                                   'calculate_rpfs_twct_worstcase.csv')
###det_file = os.path.join(resultfolder, 'PFSP_Cmax_deterministic_all_results.csv')
# Worstcase calculations for the SimGRASP methods
stoc_file = os.path.join('..', 'pfsp_experiments', 'run_simgrasp_twct', 'run_simgrasp_twct_stochgrasp_results.csv')

### Create the output folder 

In [ ]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_graph = os.path.join(os.getcwd(), 'results', 'consolidated', 'graphs')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'final')
if not os.path.exists(outputfolder_graph):
    os.makedirs(outputfolder_graph)
#print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [ ]:
# Robust PFSP Budget solutions only
df_rpfs = pd.read_pickle(rpfs_file)
df_rpfs.drop(columns=['executionId'], inplace=True)
df_rpfs = df_rpfs.reset_index()
# Robust PFSP Budget worst-case costs (budget)
df_rpfs_worstcase = pd.read_csv(rpfs_worstcase_file, delimiter=',')
df_rpfs_worstcase.drop(columns=['executionId','model','permutation'], inplace=True)
# Stochastic PFSP (SimGRASP), including worst-case costs (budget)
df_stoc = pd.read_csv(stoc_file, delimiter=',')

**Robust dataframe: calculating new fields.**

In [ ]:
df_rpfs['optimal'] = df_rpfs['is_optimal'] & df_rpfs['validated'] & (df_rpfs['gap'] <= 1e-8)
df_rpfs['time_limit'] = 7200.0
df_rpfs['time_limit_2'] = 7200.0 * 2
df_rpfs['mp_total_time'] = (df_rpfs['n'] < 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit']) + (df_rpfs['n'] >= 15).astype(int) * np.minimum(df_rpfs['mp_total_time'], df_rpfs['time_limit_2'])
df_rpfs['time'] = df_rpfs['mp_total_time'] + df_rpfs['sp_total_time']
df_rpfs['gap'] = df_rpfs['gap'] * 100.0
df_rpfs['RobCost_worstcase'] = df_rpfs['wct_validation']
df_rpfs = df_rpfs.rename(columns={"budget_Gamma": "RobCost_Gamma"})
df_rpfs.head(2)

In [ ]:
df_rpfs.info()

### Obtaining the Robust PFSP Budget robust costs (worst-case scenarios) for all RPFS solutions

For all combinations of Gamma.

In [ ]:
df_rpfs_worstcase.head(2)

In [ ]:
df_rpfs_worstcase['RobCost_Gamma'] = df_rpfs_worstcase['GammaPercRobustCost']
df_rpfs_worstcase['RobCost_worstcase'] = df_rpfs_worstcase['RobustCost']
df_rpfs_worstcase['GammaPerc'] = 100 * df_rpfs_worstcase['Gamma'] / (df_rpfs_worstcase['m'] * df_rpfs_worstcase['n'])
df_rpfs_worstcase['GammaPerc'] = df_rpfs_worstcase['GammaPerc'].astype(int)
df_rpfs_worstcase.head(2)

In [ ]:
df_rpfs_robustcost = df_rpfs_worstcase.copy()
rpfs_columns = ['n', 'm', 'alpha', 'instance_name', 'GammaPerc'] # , 'RobCost_Gamma']
df_rpfs_robustcost = df_rpfs_robustcost.sort_values(rpfs_columns)
df_rpfs_robustcost

### Analyzing the stochastic solutions dataframe (SimGRASP) 

In [ ]:
df_stoc.info()

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df_stoc.head(2))

### Obtaining the Robust PFSP Budget robust costs (worst-case scenarios) for all SimGRASP solutions

In [ ]:
df_stoc['instance_name'] = df_stoc['instance_filename']

df_ssgrasp = df_stoc.copy()
df_ssgrasp['RobCost_Gamma'] = df_ssgrasp['GammaPercRobustCost']
df_ssgrasp['RobCost_worstcase'] = df_ssgrasp['RobustCost']
df_ssgrasp

Notice we have 25 executions for each instance file (and respective alpha parameter). For result comparison, we will need one worstcase cost per instance and budget_Gamma. For now, we will group by instance file in order to obtain the smallest and largest worstcase costs found after 25 SimGRASP executions. We will call them `Min(25)` and `Max(25)`. 

In [ ]:
#df_ssgrasp_min_worstcost = df_ssgrasp.groupby(['n', 'm', 'alpha', 'instance_name', 'stochsol_exp_cost', 'stochsol_time', 
#                                               'RobCost_Gamma']).min()
ssgrasp_columns = ['n', 'm', 'alpha', 'instance_name', 'RobCost_Gamma']
df_ssgrasp_min_worstcost = df_ssgrasp[df_ssgrasp['RobCost_worstcase'] == df_ssgrasp.groupby(ssgrasp_columns)['RobCost_worstcase']
                                                                                  .transform('min')]
df_ssgrasp_min_worstcost = df_ssgrasp_min_worstcost.sort_values(ssgrasp_columns).drop_duplicates(ssgrasp_columns)
df_ssgrasp_max_worstcost = df_ssgrasp[df_ssgrasp['RobCost_worstcase'] == df_ssgrasp.groupby(ssgrasp_columns)['RobCost_worstcase']
                                                                                  .transform('max')]
df_ssgrasp_max_worstcost = df_ssgrasp_max_worstcost.sort_values(ssgrasp_columns).drop_duplicates(ssgrasp_columns)

display(df_ssgrasp_min_worstcost.tail(4))
display(df_ssgrasp_max_worstcost.tail(4))

# Tables

## Table 4. Robust price and hedge value - TODO

**Note:** the data generated in this Table depends on a post-processing with the Julia script: ``src/julia/experiment/wct/calculate_cmax_of_2rpfs_solution_given_nominal_P_bar.jl``

In [ ]:
df_det_d0 = df_dpfs[(df_dpfs['perc_deviation_p_bar'] == 0)]
df_det_hedge = df_det_d0.copy()
df_2rpfs_hedge = df_rpfs_wagner.copy()

In [ ]:
join_columns = ['n', 'm', 'alpha', 'instance_name', 'RobCost_Gamma1', 'RobCost_Gamma2']

In [ ]:
df_det_hedge = df_det_hedge.rename(columns={"cmax_star": "Cmax_sigma_det_star", "RobCost_worstcase": "RobCost_sigma_det_star"})

In [ ]:
df_det_hedge = df_det_hedge[join_columns + ['Cmax_sigma_det_star', 'RobCost_sigma_det_star']]

In [ ]:
df_2rpfs_hedge = df_2rpfs_hedge.rename(columns={'cmax_dp': 'RobCost_sigma_rob_star', 
                                                'cmax_nominal': 'Cmax_nominal_sigma_rob'})

In [ ]:
df_2rpfs_hedge = df_2rpfs_hedge[join_columns + ['RobCost_sigma_rob_star', 'Cmax_nominal_sigma_rob', 'seq']]

In [ ]:
df_det_rob = pd.merge(df_det_hedge, df_2rpfs_hedge, how='inner', on=join_columns)

In [ ]:
df_det_rob.head()

In [ ]:
df_det_rob['eta'] = df_det_rob['Cmax_nominal_sigma_rob'] - df_det_rob['Cmax_sigma_det_star']

In [ ]:
df_det_rob['H'] = df_det_rob['RobCost_sigma_det_star'] - df_det_rob['RobCost_sigma_rob_star']

In [ ]:
df_det_rob.head(25)

In [ ]:
df_det_rob.to_csv(os.path.join(outputfolder_table, '2rpfs_robustprice_hedge.csv'), sep=';')

In [ ]:
#df_det_rob[df_det_rob['instance_name'] == 'RB0205010.txt']  # 'RB2005010.txt', 'RB1502008.txt'
df_ = df_det_rob[(df_det_rob['n'] == 150) & (df_det_rob['seq'] == 8)]
piv = pd.pivot_table(df_, columns=['n', 'seq', 'RobCost_Gamma1', 'RobCost_Gamma2', 'alpha'], values=['eta', 'H'],
                       aggfunc={'eta' : ['mean'], 'H' : 'mean'})  # , margins=True, fill_value=0)
piv.to_csv(os.path.join(outputfolder_table, '2rpfs_robustprice_hedge_test.csv'), sep=';')

In [ ]:
df_det_d0[(df_det_d0['n'] == 150) & (df_det_d0['instance_name'] == 'RB1502008.txt')]

In [ ]:
df_grouped = df_rpfs_wagner.groupby(['alpha', 'n', 'm', 'RobCost_Gamma']).agg({'cmax_dp' : ['count']}).reset_index()
df_grouped.columns = [ ' '.join(str(i) for i in col) for col in df_grouped.columns]
#df_grouped.reset_index(inplace=True)
df_grouped

# Graphs

In [ ]:
join_columns = ['n', 'm', 'alpha', 'instance_name', 'Gamma']

In [ ]:
def plot_worstcase_comparison(n, instance_name, palette, df_dict, prefix, suffix, enable_markers=True):
    # https://seaborn.pydata.org/tutorial/aesthetics.html
    concat_columns = ['instance_name', 'RobCost_Gamma', 'RobCost_worstcase']
    for key, df_i in df_dict.items():
        df_i = df_i[concat_columns]
        df_i['Method'] = key
        df_dict[key] = df_i
    df = pd.concat(df_dict.values())
    ###df = df[(df['instance_name'] == instance_name)]
    #df['RobCost_Gamma'] = (df['RobCost_Gamma']).map(int)
    df['budget_f'] = df['RobCost_Gamma'].map(str)
    # https://www.drawingfromdata.com/setting-figure-size-using-seaborn-and-matplotlib
    #fig, ax = plt.subplots()
    # the size of A4 paper
    #fig.set_size_inches(11.7, 8.27)
    markers = None
    linestyles = None
    if enable_markers:
        marker = ['*', '+', 'o', 'x', '^', '8', 's', 'p', 'D',  '*', '+', 'o', 'x', '^', '8', 's', 'p', 'D']
        marker = marker[::-1]
        markers = [marker[i] for i in range(len(df["Method"].unique()))]
        linestyle = ['--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', '-.',    '--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', '-.']
        linestyles = [linestyle[i] for i in range(len(df["Method"].unique()))]
    width = 12
    height = 24
    a4_dims = (width, height)  # 8.27)
    plt.figure(figsize=a4_dims)
    with sns.axes_style("whitegrid"):
        sns.set_context("paper", font_scale=1.2, rc={"grid.linewidth": 2})
        if enable_markers:
            g = sns.catplot(x="budget_f", y="RobCost_worstcase", linestyles=linestyles, markers=markers,
                            #dashes=[linestyle_tuple[i][1][1] for i in range(len(linestyle_tuple))],
                     hue="Method", kind="point", style="Method", 
                     data=df,
                     height=height, aspect=width/height,
                     #height=5, # make the plot 5 units high
                     #aspect=3,  # 3  # height should be three times width
                     legend=False, palette=palette)
        else:
            g = sns.catplot(x="budget_f", y="RobCost_worstcase", 
                     hue="Method", kind="point", style="Method", 
                     data=df,
                     height=height, aspect=width/height,
                     #height=5, # make the plot 5 units high
                     #aspect=3,  # height should be three times width
                     legend=False, palette=palette)
        plt.legend(loc='lower right')
        # Remove the '.txt' at the end of instance_name
        instance_name = instance_name[:instance_name.rfind('.txt')]
        g.set_axis_labels(r'Protection level $\Gamma$ (%)', r'Robust cost : $Z(\sigma)$')
        plt.grid(which='major', axis='x')
        plt.grid(which='minor', axis='y')
        # ============== Special trick to make the suptitle with large backgrounds
        # register the custom style
        BoxStyle._style_list["ext"] = ExtendedTextBox
        title = plt.suptitle(prefix + ' Instance ' + instance_name + ' ' + suffix, fontsize=14, fontweight='bold', 
                             position=(.5, 1.04), backgroundcolor='#d3d3d3', color='black')
        # set the box style of the title text box to our custom box
        bb = title.get_bbox_patch()
        fig = g.fig
        def resize(event):
            # use the figure width as width of the text box 
            bb.set_boxstyle("ext", pad=0.4, width=fig.get_size_inches()[0]*fig.dpi )
        resize(None)
        # =========================================================================
        plt.show()
        g.savefig(os.path.join(outputfolder_graph, 'worstcase_{}.pdf'.format(instance_name)))
        #g.savefig(os.path.join(outputfolder_graph, 'worstcase_{}.pgf'.format(instance_name)))

### Worstcase cost : Small Uncertainty Range Instance - Example 

instance_2_9_4_wct_inputs.txt with alpha_max == 100%

In [ ]:
filename = 'instance_9_4_R100_wct_inputs.txt'  # 'instance_2_9_4_wct_inputs.txt'
simgrasp_filename = 'instance_2_9_4_wct_inputs.txt'
result_dict = dict()
include_list = [10, 30, 40, 50, 100]  # 50 == 60 == ... == 100, and 10 == 20
df_ = df_rpfs_robustcost[(df_rpfs_robustcost['instance_name'] == filename)].sort_values(['RobCost_worstcase'])
for gamma in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    if gamma == 0:
        label = 'Det'
    else:
        label = 'RPFS({})'.format(gamma)
    if gamma in include_list:
        result_dict[label] = df_[(df_['GammaPerc'] == gamma)]
result_dict['SimGRASP-Min(25)'] = df_ssgrasp_min_worstcost[df_ssgrasp_min_worstcost['instance_name'] == simgrasp_filename]
result_dict['SimGRASP-Max(25)'] = df_ssgrasp_max_worstcost[df_ssgrasp_max_worstcost['instance_name'] == simgrasp_filename]
#display(result_dict['SimGRASP-Max(25)'])
plot_worstcase_comparison(n, filename, 'Set1_r', result_dict, r'$(a)$', '', enable_markers=True)  # Set1, hsv

#### Now let's check that, except for Rob(40), all robust solutions shown above have the same value, regardless of RobCost_Gamma

In [ ]:
df_ = df_rpfs_robustcost[(df_rpfs_robustcost['instance_name'] == filename)]
df_['budget_Gamma'] = df_['Gamma'].astype(str)
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df_[df_['budget_Gamma'].isin(['60', '20', '80', '40', '100'])].sort_values(['RobCost_Gamma']))

### Worstcase cost : Large Uncertainty Range Instance - Example 

instance_1_15_5_wct_inputs.txt

In [ ]:
filename = 'instance_15_5_R50_wct_inputs.txt'
simgrasp_filename = 'instance_1_15_5_wct_inputs.txt'
n = 150
result_dict = dict()
include_list = [0, 10, 20, 30, 40, 50, 60, 70, 90, 100]
df_ = df_rpfs_robustcost[(df_rpfs_robustcost['instance_name'] == filename)]
for gamma in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    if gamma == 0:
        label = 'Det'
    else:
        label = 'RPFS({})'.format(gamma)
    if gamma in include_list:
        result_dict[label] = df_[(df_['GammaPerc'] == gamma)]
#result_dict['SimGRASP-Min(25)'] = df_ssgrasp_min_worstcost[df_ssgrasp_min_worstcost['instance_name'] == simgrasp_filename]
#result_dict['SimGRASP-Max(25)'] = df_ssgrasp_max_worstcost[df_ssgrasp_max_worstcost['instance_name'] == simgrasp_filename]
plot_worstcase_comparison(n, filename, 'hsv', result_dict, r'$(b)$', '', enable_markers=True)

#### Now let's check which robust solutions shown above have the same value, regardless of RobCost_Gamma

Before (RobCost_Gamma1, RobCost_Gamma2) = (60,20), robust solutions Rob(20,60) and Rob(40,80) are almost equivalent. They present the same robust costs, except when (RobCost_Gamma1, RobCost_Gamma2) = (20,40). 


In [ ]:
df_ = df_rpfs_robustcost[(df_rpfs_robustcost['instance_name'] == filename)]
df_['budget_Gamma'] = df_['Gamma'].astype(str)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_[df_['budget_Gamma'].isin(['60', '80'])].sort_values(['RobCost_Gamma']))

After (RobCost_Gamma1, RobCost_Gamma2) = (60,20) and beyond, all three robust solutions shown on the graph are equivalent (e.g. they present the same robust cost. 

In [ ]:
df_ = df_rpfs_robustcost[(df_rpfs_robustcost['instance_name'] == filename)]
df_['budget_Gamma'] = df_['Gamma'].astype(str)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_[df_['budget_Gamma'].isin(['40', '60', '80'])].sort_values(['RobCost_Gamma']))

In [ ]:
def get_instance_solution_info(df_rpfs, instance_name):
    return df_rpfs[(df_rpfs['instance_name'] == instance_name)][['Gamma', 'is_optimal', 
                                                                   'validated', 'gap', 'time', 'optimal']]

# Part 3. Plotting Monte Carlo Simulation results

Simulations were undertaken for 3 probability distributions: lognormal, triangular and uniform.

In [ ]:
sim_results_folder_det_rob = os.path.join(os.path.abspath('..'), 'pfsp_experiments', 'montecarlo_sim_rpfs_twct')
sim_results_folder_simgrasp = os.path.join(os.path.abspath('..'), 'pfsp_experiments', 'montecarlo_sim_rpfs_twct')
sim_results_folder_ssgrasp = os.path.join(os.path.abspath('..'), 'pfsp_experiments', 'run_simgrasp_twct')
print('[Det, Rob] Using simulation results folder: ', sim_results_folder_det_rob)
print('[SimGRASP] Using simulation results folder: ', sim_results_folder_simgrasp)
print('[SSGRASP] Using simulation results folder: ', sim_results_folder_ssgrasp)

In [ ]:
def read_simulation_result_csv_to_series(filename):
    #print('Reading file: ', filename)
    df = pd.read_csv(filename, index_col=False, header=0, names=['TWCT'])
    series = df['TWCT'] # here we convert the DataFrame into a Series
    return series

In [ ]:
def read_budget_simulation_results_to_series(root_folder, instance_name, alpha, distribution, gamma, num_iter=10000):
    folder = os.path.join(root_folder, 'robust_pfsp', distribution, 'alpha{}%'.format(alpha))
    filename = 'MCS_rob_{}_{}_{}_{}_iter{}.txt.gz'.format(gamma, instance_name, alpha, distribution, num_iter)
    filepath = os.path.join(folder, filename)
    return read_simulation_result_csv_to_series(filepath)

In [ ]:
def read_deterministic_simulation_results_to_series(root_folder, instance_name, alpha, distribution, perc_variation, num_iter=10000):
    folder = os.path.join(root_folder, 'deterministic_pfsp', distribution, 'alpha{}%'.format(alpha))
    filename = 'MCS_det{}_{}_{}_{}_iter{}.txt.gz'.format(perc_variation, instance_name, alpha, distribution, num_iter)
    filepath = os.path.join(folder, filename)
    return read_simulation_result_csv_to_series(filepath)

In [ ]:
def read_ssgrasp_raw_outputs_to_series(filepath):
    with gzip.open(filepath, 'rt') as content_file:
        content = content_file.read()
        content = content[content.find('STOCH')+5:]
        #content = content[content.find('DET')+3:content.find('STOCH')]
        content = content.replace("\n", "")
        s = pd.Series([float(x) for x in content.split()], name='TWCT')
        #s.to_csv(os.path.join(os.getcwd(), "temp.csv"), index=False, header=0)
        return s

In [ ]:
def read_stochastic_simulation_results_to_series(root_folder, instance_name, alpha, distribution, raw=True, num_iter=10000):
    if raw:  # RB0105001_10_2_t_1.0_0.1_124341_outputsList.txt
        m = 2
        n = int(instance_name[2:5])
        grasp_instance_name = 'RB{}50{}'.format(instance_name[2:5], instance_name[7:9])
        filename = '{}_{}_{}_t_{:.1f}_{:.1f}_*_outputsList.txt.gz'.format(grasp_instance_name, n, m, 1.0, alpha / 100)
        files = glob.glob(os.path.join(root_folder, filename))
        series_list = []
        for filepath in files:
            series_list.append(read_ssgrasp_raw_outputs_to_series(filepath))
        result = pd.concat(series_list)
        return result
    else:
        folder = os.path.join(root_folder, 'simgrasp', distribution, 'alpha{}%'.format(alpha))
        filename = 'MCS_SimGRASP_{}_{}_{}_iter{}.txt.gz'.format(instance_name, alpha, distribution, num_iter)
        filepath = os.path.join(folder, filename)
        return read_simulation_result_csv_to_series(filepath)

In [ ]:
def plot_violin_compare_distributions(df, instance_name, ax, seq, distribution, palette="Blues_d"):
    # https://towardsdatascience.com/violin-plots-explained-fb1d115e023d
    #a4_dims = (11.7, 8.27)
    #plt.figure(figsize=a4_dims)
    with sns.axes_style("whitegrid"):
        #style.use('ggplot')
        sns.set_context("paper", rc={"grid.linewidth": 2, "xtick.major.pad": 11})  # font_scale=1.2, 
        ax = sns.violinplot(ax=ax,y="TWCT", x="Method",   # x="TWCT", y="Method", 
                               #hue="Method", #kind="violin", 
                               style="Method", 
                     data=df, palette=palette, 
                     scale="area", cut=0, inner='box', 
                     # width=0.8, showmeans=True, showextrema=True, showmedians=True
                     height=10, # make the plot 5 units high
                     aspect=2.5) # height should be three times width
        ax.set_xticklabels(
            ax.get_xticklabels(), 
            rotation=45, 
            #horizontalalignment='right',
            fontweight='light',
            fontsize='large'
        )
        # Set Background color: https://stackoverflow.com/questions/25238442/setting-plot-background-colour-in-seaborn
        ax.set_facecolor('#f0f0f0')
        instance_name = instance_name[:instance_name.rfind('.txt')]
        ax.set_title('Simulation result from {} distribution'.format(distribution))
        if seq == 0:
            ax.set_ylabel('TWCT')
        else:
            ax.set_ylabel('')
        ax.set_xlabel('Solution Method')
        # Draw interval grid lines
        ax.grid(which='major', axis='x')
        ax.grid(which='minor', axis='y')
        #plt.set_title('Instance '+instance_name)
        #plt.show()
        #ax.get_figure().savefig(os.path.join(outputfolder_graph, 'violin_{}.pdf'.format(instance_name)))
        #display(sns.plotting_context())
        #chart.savefig(os.path.join(outputfolder_graph, 'violin_{}.pgf'.format(instance_name)))

In [ ]:
def plot_kde_compare_distributions(dict_s, instance_name, ax, seq, distribution, palette):
    #a4_dims = (11.7, 8.27)
    #plt.figure(figsize=a4_dims)
    #print('Number os series to plot: ' + str(len(dict_s.keys())))
    marker = ['*', '+', 'o', 'x', '^', '8', 's', 'p', 'D', 'V', 'A', 'T'] + ['*', '+', 'o', 'x', '^', '8', 's', 'p', 'D', 'V', 'A', 'T']
    markers = [marker[i] for i in range(len(dict_s.keys()))]
    linestyle = ['--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', ' ', '', 'None'] + ['--', '-.', ':', 'dashed', 'dashdot', 'dotted', 'solid', '-', ' ', '', 'None']
    # New line styles > 12
    # https://stackoverflow.com/questions/33337989/how-to-draw-more-type-of-lines-in-matplotlib/33338727
    linestyles = [linestyle[i] for i in range(len(dict_s.keys()))]
    with sns.axes_style("whitegrid"):
        #style.use('ggplot')
        #sns.set_style("whitegrid")
        sns.set(style="white", palette=palette) #, color_codes=True)
        #sns.set_palette(palette, 9, 0.8)
        sns.set_context("notebook", rc={"grid.linewidth": 2, "xtick.major.pad": 11})  # font_scale=1.2, 
        i = 0
        for method, series in dict_s.items():
            chart = sns.distplot(series, label=method, kde=True, hist=False, ax=ax, 
                         vertical=False, kde_kws=dict(ls=linestyle[i],dashes=linestyle_tuple[i][1][1]))  
            # markers=markers, 
            i += 1
        # Set Background color: https://stackoverflow.com/questions/25238442/setting-plot-background-colour-in-seaborn
        #chart.set_facecolor('#ffffff')
        ax.set_title('Simulation result from {} distribution'.format(distribution))
        ax.legend(loc='upper right', bbox_to_anchor=(1.10, 1.0), ncol=1)
        all_fonts_size = 12
        if seq == 0:
            ax.set_ylabel('Probability Distribution', fontsize=all_fonts_size)
        ax.set_xlabel('TWCT', fontsize=all_fonts_size)
        ax.tick_params(labelsize=all_fonts_size)
        ax.grid(which='major', axis='x')
        ax.grid(which='minor', axis='y')
        #plt.show()
        #chart.get_figure().savefig(os.path.join(outputfolder_graph, 'kde_{}.pdf'.format(instance_name)))
        #chart.savefig(os.path.join(outputfolder_graph, 'kde_{}.pgf'.format(instance_name)))

In [ ]:
def plot_simulation_comparison(n, m, sim_results_folder_det_rob, sim_results_folder_simgrasp, instance_name, 
                               simgrasp_instance_name, instance_title, 
                               prefix, suffix, 
                               palette, include_list=[], graph_type="violin", num_iter=10000):
    # https://towardsdatascience.com/violin-plots-explained-fb1d115e023d
    a4_dims = (15, 5)
    #plt.figure(figsize=a4_dims)
    fig, axs = plt.subplots(1, 3, sharey=True, figsize=a4_dims)
    #sns.set_style("whitegrid")
    for d, distribution in enumerate(['lognormal', 'uniform', 'triangular']):
        ###alpha = int(instance_name[5:7])
        alpha = 100
        simulated_solutions_dict = dict()
        
        for gamma_perc in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
            if gamma_perc == 0:
                label = 'Det'
            else:
                label = 'RPFS({})'.format(gamma_perc)
            gamma = gamma_perc * m * n / 100.0
            simulated_solutions_dict[label] = read_budget_simulation_results_to_series(
                                                                                sim_results_folder_det_rob, 
                                                                                filename, alpha, distribution, gamma,
                                                                                num_iter=num_iter)
        #simulated_solutions_dict['SimGRASP(MCS_Java)'] = read_stochastic_simulation_results_to_series(sim_results_folder_ssgrasp, 
        #                                                                                    instance_name, alpha, 
        #                                                                                    distribution, True)
        simulated_solutions_dict['SimGRASP'] = read_stochastic_simulation_results_to_series(sim_results_folder_simgrasp, 
                                                                                            simgrasp_instance_name, 100, 
                                                                                            distribution, False,
                                                                                            num_iter=num_iter)
        df_list = []
        filtered_solutions_dict = dict()
        for key, s_i in simulated_solutions_dict.items():
            df_i = s_i.to_frame()
            df_i['Method'] = key
            df_i['Distribution'] = distribution
            if key in include_list:
                df_list.append(df_i)
                filtered_solutions_dict[key] = s_i
        df = pd.concat(df_list)
        if graph_type == 'violin':
            plot_violin_compare_distributions(df, instance_name, axs[d], d, distribution, palette=palette)
        else:  # kde plot
            plot_kde_compare_distributions(filtered_solutions_dict, instance_name, axs[d], d, distribution, palette=palette)
        #axs[d].yaxis.tick_right()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    instance_name = instance_name[:instance_name.rfind('.txt')]
    # ============== Special trick to make the suptitle with large backgrounds
    # register the custom style
    BoxStyle._style_list["ext"] = ExtendedTextBox
    title = plt.suptitle(prefix + ' Instance ' + instance_title + ' ' + suffix, fontsize=12, fontweight='bold', 
                         position=(.5, 0.99), backgroundcolor='#d3d3d3', color='black')
    # set the box style of the title text box to our custom box
    bb = title.get_bbox_patch()
    def resize(event):
        # use the figure width as width of the text box 
        bb.set_boxstyle("ext", pad=0.4, width=fig.get_size_inches()[0]*fig.dpi )
    resize(None)
    # =========================================================================
    if graph_type == 'violin':
        fig.savefig(os.path.join(outputfolder_graph, 'violinplot_{}.pdf'.format(instance_name)))
    else:  # kde plot
        fig.savefig(os.path.join(outputfolder_graph, 'kdeplot_{}.pdf'.format(instance_name)))

### Expected cost : Small Uncertainty Range Instance - Example 

instance_2_9_4_wct_inputs.txt

TODO Traçar uma linha dentro do gráfico do violin plot, demarcando o valor do worst-case TWCT (dado pelo robusto)

Agora vamos traçar um gráfico de KDE com a distribuição de valores de cada simulação.

In [ ]:
filename = 'instance_9_4_R30_wct_inputs.txt'
simgrasp_filename = 'instance_2_9_4_wct_inputs.txt'
all_gamma = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
include_list = ['Det', 'SimGRASP'] + ['RPFS({})'.format(_) for _ in all_gamma]
plot_simulation_comparison(9, 4, sim_results_folder_det_rob, sim_results_folder_det_rob, filename, simgrasp_filename,
                           '9x4', r'$(a)$', '',
                           graph_type='kde',
                           include_list=include_list,
                           palette='Paired_r') # Dark2, Set3

**Analysis:** 

Regarding the small uncertainty instance, the expected TWCT performance of 2RPFS(40,60), as well as 2RPFS(40,80), 2RPFS(60,40), 2RPFS(80,40) and 2RPFS(80,80), are really good, equivalent to SimGRASP. However, the protection against worst-case scenarios vary considerably, according to the budget parameters. The best performing solutions, from smallest to largest robust cost, are: 2RPFS(80,40), 2RPFS(60,40), SimGRASP-Min(25), 2RPFS(40,60), 2RPFS(40,80) and 2RPFS(80,80). 
%, protect much more against worst-case costs. 
In particular, even though 2RPFS(40,20) has very good expected TWCT values, its $\Gamma$ parameter combination does not offer the best protection against worst-case costs, when compared to other robust solutions. 

### Expected cost : Large Uncertainty Range Instance - Example 

instance_15_5_R30_wct_inputs

In [ ]:
filename = 'instance_15_5_R30_wct_inputs.txt'
simgrasp_filename = 'instance_1_15_5_wct_inputs.txt'
all_gamma = [10, 20, 30, 40, 50, 60, 90, 100]  # 70, 80, 
include_list = ['Det', 'SimGRASP'] + ['RPFS({})'.format(_) for _ in all_gamma]
plot_simulation_comparison(15, 5, sim_results_folder_det_rob, sim_results_folder_det_rob, filename, simgrasp_filename, 
                           '15x5', r'$(b)$', '',
                           graph_type='kde',
                          include_list=include_list,
                          palette='Paired_r')   # Paired, hsv, ***Set1

When analyzing the large uncertainty instance, several robust solutions present expected TWCT performance quite similar to SimGRASP: 2RPFS(40,20), 2RPFS(40,40), 2RPFS(60,40), 2RPFS(60,60) and 2RPFS(100,80). However, only the first four provide better protection against worst-case costs. Even though 2RPFS(100,80) presents very good expected TWCT values, its robust costs (worst-case performance) is suboptimal for several values of $\Gamma_{1}$ and $\Gamma_{2}$, as can be seen in Figure x(b), from the previous analysis. 
In the present graph, we also include the results from the solution method with the highest expected TWCT value: Rob(20,80). 

# Monte Carlo Simulation results - Summary table

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
def table_simulation_comparison(n, m, sim_results_folder_det_rob, sim_results_folder_ssgrasp, instance_name, 
                                simgrasp_instance_name, 
                                prefix, suffix, 
                                save_csv=False, include_results=None, num_iter=10000):
    grouped_df_list = []
    grouped_df_list_tableau = []
    for d, distribution in enumerate(['lognormal','normal', 'uniform', 'triangular']):
        ###alpha = int(instance_name[5:7])
        alpha = 100
        simulated_solutions_dict = dict()
        for gamma_perc in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
            if gamma_perc == 0:
                label = 'Det'
            else:
                label = 'RPFS({})'.format(gamma_perc)
            gamma = gamma_perc * m * n / 100.0
            simulated_solutions_dict[(label, gamma)] = read_budget_simulation_results_to_series(
                                                                            sim_results_folder_det_rob, 
                                                                            instance_name, alpha, distribution, gamma,
                                                                            num_iter=num_iter)
        simulated_solutions_dict[('SimGRASP', 200, 200)] = read_stochastic_simulation_results_to_series(sim_results_folder_simgrasp, 
                                                                                            simgrasp_instance_name, alpha, 
                                                                                            distribution, False,
                                                                                            num_iter=num_iter)
        #simulated_solutions_dict[('SimGRASP(MCS_Java)', 200, 200)] = read_stochastic_simulation_results_to_series(sim_results_folder_ssgrasp, 
        #                                                                                    instance_name, alpha, 
        #                                                                                    distribution, True)
        df_list = []
        filtered_solutions_dict = dict()
        for key, s_i in simulated_solutions_dict.items():
            df_i = s_i.to_frame()
            df_i['Method'] = key[0]
            df_i['Gamma'] = key[1]
            df_i['Distribution'] = distribution
            if include_results is None or key[0] in include_results:
                df_list.append(df_i)
                filtered_solutions_dict[key] = s_i
        df = pd.concat(df_list)
        df_grouped = df.copy()
        df_grouped = df_grouped.groupby(['Method', 'Distribution']).agg({'TWCT':[np.mean, np.std, np.max, percentile(95), \
                                                                             percentile(99), 'count']})
        df_grouped.columns = df_grouped.columns.droplevel()
        df_grouped = df_grouped.reset_index()
        value_columns = ['mean', 'std', 'percentile_95', 'percentile_99', 'amax', 'count']
        # Melt the value columns -> key, value style
        df_grouped = pd.melt(df_grouped, id_vars=['Method', 'Distribution'], value_vars=value_columns)
        df_grouped['instance'] = instance_name
        # For each variable, mark the smallest value (among all methods) with a binary marker in an extra column
        df_grouped['is_min'] = df_grouped['value'] == df_grouped.groupby(['Distribution', 'variable'])['value'] \
                                                                                  .transform('min')
        #print(df_grouped)
        #df_grouped.columns = df_grouped.columns.droplevel(0)
        grouped_df_list_tableau.append(df_grouped)
        #df_grouped.rename(columns=dict(zip(df_grouped.columns[[0, 1, 2, 3, 4]], labels)),inplace=True)
        
        df = df.rename(columns={"TWCT": distribution})
        table = pd.pivot_table(df, values=[distribution], index=['Gamma', 'Method'], columns=['Distribution'], \
                           aggfunc={distribution: [np.mean, np.std, np.max, percentile(95), percentile(99),'count']})
        table.columns = table.columns.droplevel(2)
        # Re-order table columns
        column_order = [(distribution,'mean'), (distribution,'std'), (distribution,'percentile_95'), \
                        (distribution,'percentile_99'), (distribution,'amax'), (distribution,'count')]
        table = table.reindex(column_order, axis=1)
        # Renaming the pivot table columns to improve presentation
        #labels = [r'$E(\varphi(\sigma))$', r'$SD(\varphi(\sigma))$', r'$\varphi_{0.95}(\sigma)$', \
        #          r'$\varphi_{0.99}(\sigma)$', r'$\varphi_{max}(\sigma)$']
        #table = table.rename(columns={"mean": labels[0], "std": labels[1], "percentile_95": labels[2], \
        #                              'percentile_99': labels[3], 'amax': labels[4]}, level=1, errors="raise")
        table.columns.set_levels(['Results from {} distribution'.format(distribution)],level=0,inplace=True)
        table = table.round(1)
        grouped_df_list.append(table.copy())
        #print(table)
    # end for
    df_to_tableau = pd.concat(grouped_df_list_tableau)
    df_all = pd.concat(grouped_df_list, join='inner', axis=1)
    print(df_all)
    #print(df_all.to_latex(escape=False, float_format='%.1f', sparsify=True, label='tab:simresults_{}'.format(instance_name), 
    #                      caption='Summary of the simulation for deterministic, 2RPFS and SimGRASP solution methods' + \
    #                      ' from lognormal, triangular and uniform distributions.'))
    if save_csv:
        df_all.to_csv(os.path.join(outputfolder, 'table_simulation_results_{}.csv'.format(instance_name)), sep=';')
        df_to_tableau.to_csv(os.path.join(outputfolder, 'stats_simulation_results_{}.csv'.format(instance_name)), sep=';', index=False)
    #sup_labels = ['Results from {} distribution'.format(dist) for dist in ['lognormal', 'symmetric triangular', 'uniform']]
    

### Small uncertainty range instance

In [ ]:
filename = 'instance_2_9_4_wct_inputs.txt'
simgrasp_filename = 'instance_2_9_4_wct_inputs.txt'
table_simulation_comparison(9, 4, sim_results_folder_det_rob, sim_results_folder_ssgrasp, filename, simgrasp_filename,
                            r'$(a)$', '',
    #include_results=['2RPFS(100,100)', '2RPFS(100,60)', '2RPFS(100,80)', '2RPFS(20,20)', '2RPFS(20,40)', '2RPFS(40,40)', \
    #'2RPFS(40,60)', '2RPFS(40,80)', '2RPFS(60,20)', '2RPFS(60,40)', '2RPFS(60,80)', '2RPFS(80,40)', '2RPFS(80,80)', \
    #'Det', 'SimGRASP'], 
    save_csv=True, num_iter=10000)

### Large uncertainty range instance

In [ ]:
filename = 'instance_15_5_R30_wct_inputs.txt'
simgrasp_filename = 'instance_1_15_5_wct_inputs.txt'
table_simulation_comparison(15, 5, sim_results_folder_det_rob, sim_results_folder_ssgrasp, filename, simgrasp_filename, 
                            r'$(b)$', '',
                           #include_results=['2RPFS(100,80)', '2RPFS(20,20)', '2RPFS(20,40)', '2RPFS(40,20)', '2RPFS(40,40)', \
                           # '2RPFS(40,60)', '2RPFS(60,40)', '2RPFS(60,60)', '2RPFS(60,80)', '2RPFS(80,40)', '2RPFS(20,80)', \
                           # 'Det', 'SimGRASP'], 
                            save_csv=True) # , num_iter=1000000)